**Задание 1.**

Изучите технологии attention и архитектуры нейронных сетей трансформеров.

**Технология внимания (attention)** в нейронных сетях является ключевым компонентом, который позволяет моделям фокусироваться на определенных частях входных данных с различными весами. Она активно используется в различных областях машинного обучения, включая обработку естественного языка и компьютерное зрение. Механизм внимания может принимать разные формы, но общая идея заключается в том, чтобы в каждый момент времени модель могла "сосредотачиваться" на определенных аспектах входных данных.

**Архитектура трансформеров** является одной из наиболее влиятельных и успешных архитектур в области глубокого обучения. Она была представлена в статье "Attention is All You Need" в 2017 году и предложила новый подход к обработке последовательностей с использованием многоголового механизма внимания. Основные черты трансформера включают в себя:

1. **Многоголовое внимание:** Вместо того чтобы использовать одно внимание, трансформер использует несколько "голов" внимания, каждая из которых обучается фокусироваться на разных частях входных данных. Результаты от всех голов объединяются для получения финального выхода.

2. **Позиционное кодирование:** Поскольку трансформеры не обладают встроенным представлением порядка в последовательности, им нужно как-то учитывать порядок слов. Для этого используется позиционное кодирование, которое добавляет информацию о позиции каждого элемента в последовательности.

3. **Блоки кодировщика и декодера:** Архитектура трансформера состоит из нескольких блоков кодировщика и декодера. Каждый блок включает в себя слой многоголового внимания, пропуск соединений, перцептрон и нормализацию по слою.

4. **Само внимание (Self-Attention):** Механизм внимания в трансформерах позволяет модели уделять внимание различным частям входных данных в зависимости от контекста. В случае само внимания (self-attention), модель может учитывать относительные важности различных элементов внутри одной последовательности.

5. **Применение в различных задачах:** Трансформеры успешно применяются в широком спектре задач, таких как машинный перевод, обработка естественного языка, генерация текста, компьютерное зрение и даже в аудиообработке.

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from transformers import AdamW
from torch.nn.parallel import DistributedDataParallel

**Задание 2**

Примените один из трансформеров, например BERT к
задаче классификации отзывов клиентов. Сравните полученные результаты с
классическими методами машинного обучения, с RNN. Сделайте выводы.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/dataset.xlsx', header=None, usecols=[3, 4], names=['rating', 'review'])
data = data.dropna(subset=['rating'])
data['rating'] = data['rating'].astype(int)
data.head(5)

,rating,review
0,4,"Очень понравился ресторан, хочется его рекомен..."
1,4,"Душевный ресторан, очень подходит для романтич..."
2,4,Хочу поблагодарить всю команду зала Добролюбоф...
3,4,"Плов 10 из 10, в первый раз попробовала манты ..."
4,2,Отвратительное обслуживание Шоколадница на Лен...


In [ ]:
data['review'][1]

'Душевный ресторан, очень подходит для романтического ужина. Атмосфера уюта, всё по домашнему, обслуживание и кухня на высоте профессионализма. Спасибо за прекрасный вечер. Очень рекомендую.'

In [ ]:
device = 'cuda'

In [ ]:
data['rating'].unique()

array([4, 2, 3, 5, 1])

In [ ]:
dataset = pd.DataFrame(columns=['review', 'rating'])

for rating_value in data['rating'].unique():
    temp = data[data['rating'] == rating_value].sample(n=300, random_state=15, replace=True)
    dataset = dataset.append(temp, ignore_index=True)

<ipython-input-8-393b4956590b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(temp, ignore_index=True)
<ipython-input-8-393b4956590b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(temp, ignore_index=True)
<ipython-input-8-393b4956590b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(temp, ignore_index=True)
<ipython-input-8-393b4956590b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(temp, ignore_index=True)
<ipython-input-8-393b4956590b>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [ ]:
review = dataset['review'].tolist()
rating = dataset['rating'].tolist()

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(review, rating, test_size=0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to(device)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=100, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=100, return_tensors='pt')

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

In [ ]:
def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0.0
    total = 0.0
    correct = 0.0

    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        labels -= 1
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
        total += labels.shape[0]
        _, prediction_indices = torch.max(outputs.logits, 1)
        correct += torch.sum(prediction_indices==labels)

    train_accuracy = float(correct) / total

    return total_loss, train_accuracy

In [ ]:
def test(model, test_loader):
    model.eval()
    total_loss = 0.0
    total = 0.0
    correct = 0.0
    with torch.no_grad():
        for input_ids, attention_mask, labels in test_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            labels -= 1
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss
            total += labels.shape[0]
            _, predictions_indices = torch.max(outputs.logits, 1)
            correct += torch.sum(predictions_indices==labels)

    test_accuracy = correct / total
    return total_loss, test_accuracy

In [ ]:
losses_train = []
losses_test = []

In [ ]:
for epoch in range(20):
    torch.cuda.empty_cache()
    train_loss, train_accuracy = train(model, train_loader, optimizer)
    losses_train.append(train_loss.item())

    test_loss, test_accuracy = test(model, test_loader)
    losses_test.append(test_loss.item())

    print('Epoch: ', epoch+1)
    print(f'Loss: {train_loss:.4f}, train accuracy: {train_accuracy:.4f}')
    print(f'Loss: {test_loss:.4f}, test accuracy: {test_accuracy:.4f}')

Epoch:  1
Loss: 16.2371, train accuracy: 0.1750
Loss: 4.8323, test accuracy: 0.1867
Epoch:  2
Loss: 16.2059, train accuracy: 0.2033
Loss: 4.8225, test accuracy: 0.2233
Epoch:  3
Loss: 16.0724, train accuracy: 0.2050
Loss: 4.8439, test accuracy: 0.1633
Epoch:  4
Loss: 16.0067, train accuracy: 0.2375
Loss: 4.8131, test accuracy: 0.2033
Epoch:  5
Loss: 16.0045, train accuracy: 0.2208
Loss: 4.7781, test accuracy: 0.2233
Epoch:  6
Loss: 15.8974, train accuracy: 0.2367
Loss: 4.7207, test accuracy: 0.2500
Epoch:  7
Loss: 15.5109, train accuracy: 0.2867
Loss: 4.6158, test accuracy: 0.3067
Epoch:  8
Loss: 14.3441, train accuracy: 0.3825
Loss: 4.0680, test accuracy: 0.4567
Epoch:  9
Loss: 11.9124, train accuracy: 0.5017
Loss: 3.3893, test accuracy: 0.5467
Epoch:  10
Loss: 9.4147, train accuracy: 0.6117
Loss: 3.2919, test accuracy: 0.5767
Epoch:  11
Loss: 8.2259, train accuracy: 0.6533
Loss: 2.5935, test accuracy: 0.6433
Epoch:  12
Loss: 6.8247, train accuracy: 0.7408
Loss: 2.5034, test accuracy:

In [ ]:
import torch
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2

In [ ]:
pip install pymorphy2

In [ ]:
stemmer = SnowballStemmer("russian")
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[a-zA-Z]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [morph.parse(word)[0].normal_form for word in tokens if word not in string.punctuation]
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
example_review = 'Душевный ресторан, очень подходит для романтического ужина. Атмосфера уюта, всё по домашнему, обслуживание и кухня на высоте профессионализма. Спасибо за прекрасный вечер. Очень рекомендую.'
example = preprocess(example_review)
example = tokenizer(example, truncation=True, padding=True, max_length=250, return_tensors='pt')

In [ ]:
model.eval()
with torch.no_grad():
    inputs = {'input_ids': example['input_ids'], 'attention_mask': example['attention_mask']}
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model(**inputs)

predicted_rating = torch.argmax(outputs.logits, dim=1).item() + 1

print(f"Отзыв: {example_review}")
print(f"Рейтинг: {predicted_rating}")

Отзыв: Душевный ресторан, очень подходит для романтического ужина. Атмосфера уюта, всё по домашнему, обслуживание и кухня на высоте профессионализма. Спасибо за прекрасный вечер. Очень рекомендую.
Рейтинг: 4


Если сравнивать эта работу, где мы используем трансформер Bert, и прошлую работу, где мы используем RNN, то можем сделать такие выводы:

1. BERT является трансформерной архитектурой, которая обладает способностью эффективно улавливать контекстные зависимости в тексте. BERT способен анализировать текст в обоих направлениях (bidirectional), что делает его мощным инструментом для понимания контекста. RNN обрабатывает текст последовательно и имеет ограниченную способность улавливать дальние зависимости в тексте из-за проблемы затухания градиентов.

2. У bert вычислительные ресурсы гораздо больше и требуется больше времени для обучения, нежели rnn.

3. BERT эффективно учитывает контекст, включая зависимости между словами в предложении. Модель способна понимать семантические отношения между словами в обоих направлениях. RNN обрабатывает текст последовательно, учитывая только предыдущие слова при анализе текущего. Может столкнуться с проблемами в обработке долгосрочных зависимостей.

**Задание 3**

Примените один из трансформеров, например BERT, к
задаче генерации англоязычного и русскоязычного текстов. Сравните
результаты с LSTM. Сделайте выводы.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
input_text = "Don't stop the music"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
attention_mask = torch.ones(input_ids.shape)

output = model.generate(
    input_ids, # Тензор идентификаторов токенов, представляющих входные данные для генерации.
    max_length=100,
    attention_mask=attention_mask, # Тензор внимания, указывающий, какие токены модель должна учитывать (значение 1) и на какие токены можно игнорировать (значение 0).
    num_beams=5, # Количество лучей в алгоритме поиска лучших последовательностей.
    no_repeat_ngram_size=2,
    top_k=50, # Верхнее количество наиболее вероятных токенов, которые модель рассматривает при генерации.
    top_p=0.95, # Верхнее значение кумулятивной вероятности для токенов.
    temperature=0.7, # Коэффициент температуры, который влияет на разнообразие генерации.
    do_sample=True, #  Флаг, указывающий, используется ли стратегия выбора образца при генерации.
    pad_token_id=model.config.eos_token_id, # Идентификатор токена, используемого для паддинга. В вашем случае установлен в model.config.eos_token_id, что означает, что для паддинга используется идентификатор конечного символа последовательности (eos).
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Пример: ", input_text)
print("Сгенерированный текст: ", generated_text)

Пример:  Don't stop the music
Сгенерированный текст:  Don't stop the music, it's not going to make you happy."

"I don't know if I'll ever be able to do it again, but I've got a lot of work ahead of me," he said. "I'm not sure I'm ready for the next step. It's just a matter of time before I get to that point."


In [ ]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

In [ ]:
input_text = "Буря мглою небо кроет"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

#output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7, do_sample=True)

output = model.generate(
    input_ids, # Тензор идентификаторов токенов, представляющих входные данные для генерации.
    max_length=100,# Тензор внимания, указывающий, какие токены модель должна учитывать (значение 1) и на какие токены можно игнорировать (значение 0).
    num_beams=5, # Количество лучей в алгоритме поиска лучших последовательностей.
    no_repeat_ngram_size=2,
    top_k=50, # Верхнее количество наиболее вероятных токенов, которые модель рассматривает при генерации.
    top_p=0.95, # Верхнее значение кумулятивной вероятности для токенов.
    temperature=0.7, # Коэффициент температуры, который влияет на разнообразие генерации.
    do_sample=True, #  Флаг, указывающий, используется ли стратегия выбора образца при генерации.
    pad_token_id=model.config.eos_token_id, # Идентификатор токена, используемого для паддинга. В вашем случае установлен в model.config.eos_token_id, что означает, что для паддинга используется идентификатор конечного символа последовательности (eos).
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Пример: ", input_text)
print("Сгенерированный текст: ", generated_text)

Пример:  Буря мглою небо кроет
Сгенерированный текст:  Буря мглою небо кроет,
		И, как в былые дни, —
		                                                Вновь и вновь.

   	В. Маяковский. «Борис Годунов». «Двенадцать стульев» (1905)
Стихотворение написано в 1906 г. в Москве, в доме, где жил Маяковский, и посвящено Борису Годунову. Оно было опубликовано в журнале «


Если сравнивать эта работу, где мы используем модель трансформер GPT-2, и прошлую работу, где мы используем LSTM (тип RNN), то можем сделать такие выводы:

1. LSTM имеет ограничения в обработке длинных зависимостей, а GPT-2 обучается на более широких контекстах и способен генерировать качественные и связные тексты. Соостветсвенно GPT-2 генерирует текст более качественно.

2. LSTM требует меньше вычислительных ресурсов и может быть обучена на более небольших данных, обучение GPT-2 требует значительных вычислительных ресурсов и объема данных.

3. LSTM обрабатывает контекст на уровне последовательностей, но может столкнуться с проблемами в долгосрочном запоминании. GPT-2 использует механизм внимания для эффективного учета контекста, что делает его более способным улавливать зависимости в данных.